In [13]:

import os
import json
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# GPU setup
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs detected: {len(gpus)}")
    else:
        print("No GPU detected, using CPU")
except Exception as e:
    print("GPU setup note:", e)

No GPU detected, using CPU


In [15]:
# ============================================================================
# IMPROVED CONFIGURATION
# ============================================================================
BATCH_SIZE = 32  # Increased back to 32 for faster training
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
EPOCHS = 50
LEARNING_RATE = 3e-4  # Increased initial learning rate
VALIDATION_SPLIT = 0.2
SEED = 42

In [16]:
# Paths
BASE_DIR = "archive/dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VALID_DIR = os.path.join(BASE_DIR, "valid")
TEST_DIR = os.path.join(BASE_DIR, "test")
CAT_TO_NAME_JSON = "archive/cat_to_name.json"

MODELS_DIR = "models"
Path(MODELS_DIR).mkdir(parents=True, exist_ok=True)

In [17]:

# ============================================================================
# DATA LOADING AND INSPECTION
# ============================================================================
print("Inspecting dataset structure...")

# Check if directories exist
for dir_path, name in [(TRAIN_DIR, "Train"), (VALID_DIR, "Valid"), (TEST_DIR, "Test")]:
    if os.path.exists(dir_path):
        if name in ["Train", "Valid"]:
            subdirs = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]
            print(f"{name} directory: {len(subdirs)} classes found")
        else:
            files = [f for f in os.listdir(dir_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            print(f"{name} directory: {len(files)} images found")
    else:
        print(f"WARNING: {dir_path} not found!")

# Load class names
with open(CAT_TO_NAME_JSON, 'r') as f:
    cat_to_name = json.load(f)

# Get actual class directories from train folder
actual_class_dirs = sorted([d for d in os.listdir(TRAIN_DIR) 
                           if os.path.isdir(os.path.join(TRAIN_DIR, d))], 
                          key=lambda x: int(x) if x.isdigit() else x)

print(f"Found {len(actual_class_dirs)} classes: {actual_class_dirs[:10]}...")

num_classes = len(actual_class_dirs)
print(f"Total classes: {num_classes}")

Inspecting dataset structure...
Train directory: 102 classes found
Valid directory: 102 classes found
Test directory: 819 images found
Found 102 classes: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']...
Total classes: 102


In [18]:
# ============================================================================
# IMPROVED DATA AUGMENTATION
# ============================================================================
def create_data_generators():
    # More conservative augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,  # Reduced from 45
        width_shift_range=0.15,  # Reduced from 0.25
        height_shift_range=0.15,  # Reduced from 0.25
        shear_range=0.15,  # Reduced from 0.25
        zoom_range=0.2,  # Reduced from 0.3
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],  # More conservative
        fill_mode='nearest',
        # Add channel shift for color variation
        channel_shift_range=20.0
    )
    
    # Validation and test: only rescaling
    valid_test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        seed=SEED
    )
    
    validation_generator = valid_test_datagen.flow_from_directory(
        VALID_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, validation_generator, valid_test_datagen

train_gen, val_gen, test_datagen = create_data_generators()

print(f"Train samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Classes found: {len(train_gen.class_indices)}")

Found 6552 images belonging to 102 classes.
Found 818 images belonging to 102 classes.
Train samples: 6552
Validation samples: 818
Classes found: 102


In [19]:
#============================================================================
# CLASS WEIGHTS CALCULATION
# ============================================================================
def calculate_class_weights(generator):
    class_indices, class_counts = np.unique(generator.classes, return_counts=True)
    weights_arr = compute_class_weight(
        class_weight='balanced',
        classes=class_indices,
        y=generator.classes
    )
    class_weights = {int(c): float(w) for c, w in zip(class_indices, weights_arr)}
    
    print(f"Class weight range: {min(class_weights.values()):.3f} - {max(class_weights.values()):.3f}")
    return class_weights

class_weights = calculate_class_weights(train_gen)

Class weight range: 0.312 - 2.379


In [20]:
# ============================================================================
# IMPROVED MODEL ARCHITECTURE
# ============================================================================
def build_improved_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    """
    Improved CNN with better architecture and regularization
    """
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        
        # Block 1
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 5 - Additional depth
        layers.Conv2D(512, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.GlobalAveragePooling2D(),
        
        # Dense layers
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def build_transfer_learning_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    """
    Transfer learning model using ResNet50 (more stable than EfficientNet)
    """
    base_model = keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

In [21]:
# ============================================================================
# DIAGNOSTIC CHECKS
# ============================================================================
print("Running diagnostic checks...")

# Check if generators are working properly
sample_batch = next(train_gen)
print(f"Batch shape: {sample_batch[0].shape}")
print(f"Label shape: {sample_batch[1].shape}")
print(f"Image value range: [{sample_batch[0].min():.3f}, {sample_batch[0].max():.3f}]")
print(f"Labels sum check: {sample_batch[1].sum(axis=1)[:5]}")  # Should all be 1.0

# Verify class mapping
print(f"Generator found {len(train_gen.class_indices)} classes")
print("First few class indices:", dict(list(train_gen.class_indices.items())[:5]))

# Choose model architecture - START WITH SIMPLER MODEL
USE_TRANSFER_LEARNING = True  # Set to False for CNN from scratch
USE_SMALLER_MODEL = False      # Start with smaller model for debugging

def build_simple_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    """Simple CNN for debugging"""
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

if USE_SMALLER_MODEL:
    model = build_simple_cnn()
    base_model = None
    print("Using Simple CNN for debugging")
elif USE_TRANSFER_LEARNING:
    model, base_model = build_transfer_learning_model()
    print("Using Transfer Learning with EfficientNetB0")
else:
    model = build_improved_cnn()
    base_model = None
    print("Using Custom CNN")

model.summary()

Running diagnostic checks...
Batch shape: (32, 224, 224, 3)
Label shape: (32, 102)
Image value range: [0.000, 1.000]
Labels sum check: [1. 1. 1. 1. 1.]
Generator found 102 classes
First few class indices: {'1': 0, '10': 1, '100': 2, '101': 3, '102': 4}
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Using Transfer Learning with EfficientNetB0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 102)            │        52,326 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,699,366 (94.22 MB)

 Trainable params: 1,106,534 (4.22 MB)

 Non-trainable params: 23,592,832 (90.00 MB)

In [22]:
# ============================================================================
# IMPROVED TRAINING SETUP
# ============================================================================
# Simpler learning rate for initial debugging
initial_learning_rate = 1e-3  # Increased learning rate
optimizer = Adam(learning_rate=initial_learning_rate)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', 
             keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc'),
             keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_acc')]
)

# Callbacks
checkpoint_path = os.path.join(MODELS_DIR, 'flower_classifier_best.keras')
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',  # Monitor accuracy instead of loss
        patience=15,
        restore_best_weights=True,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

In [23]:
# ============================================================================
# TRAINING
# ============================================================================
print("Starting training...")

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

Starting training...


c:\Users\Ammad\Documents\Projects\Personal\UEL\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 10/205 ━━━━━━━━━━━━━━━━━━━━ 3:53 1s/step - accuracy: 0.0152 - loss: 6.3166 - top3_acc: 0.0409 - top5_acc: 0.0846 

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# FINE-TUNING (if using transfer learning)
# ============================================================================
if USE_TRANSFER_LEARNING and base_model is not None:
    print("\nStarting fine-tuning phase...")
    
    # Unfreeze the base model
    base_model.trainable = True
    
    # Fine-tune from this layer onwards
    fine_tune_at = len(base_model.layers) // 2
    
    # Freeze all the layers before fine_tune_at
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Use a lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=initial_learning_rate/10),
        loss='categorical_crossentropy',
        metrics=['accuracy', 
                 keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc'),
                 keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_acc')]
    )
    
    # Fine-tune
    fine_tune_epochs = 20
    total_epochs = len(history.history['accuracy']) + fine_tune_epochs
    
    history_fine = model.fit(
        train_gen,
        epochs=total_epochs,
        initial_epoch=len(history.history['accuracy']),
        validation_data=val_gen,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    # Combine histories
    for key in history.history.keys():
        history.history[key].extend(history_fine.history[key])

print('Training complete!')

In [ ]:

# ============================================================================
# EVALUATION AND VISUALIZATION
# ============================================================================
def plot_training_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Top-3 Accuracy
    if 'top3_acc' in history.history:
        axes[1, 0].plot(history.history['top3_acc'], label='Train Top-3 Acc')
        axes[1, 0].plot(history.history['val_top3_acc'], label='Val Top-3 Acc')
        axes[1, 0].set_title('Top-3 Accuracy')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Accuracy')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate
    if hasattr(model.optimizer, 'learning_rate'):
        try:
            lr_values = [model.optimizer.learning_rate(step).numpy() 
                        for step in range(len(history.history['loss']))]
            axes[1, 1].plot(lr_values)
            axes[1, 1].set_title('Learning Rate')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Learning Rate')
            axes[1, 1].set_yscale('log')
            axes[1, 1].grid(True, alpha=0.3)
        except:
            axes[1, 1].text(0.5, 0.5, 'Learning Rate\nSchedule Active', 
                           ha='center', va='center', transform=axes[1, 1].transAxes)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

# Load best model and evaluate
best_model = keras.models.load_model(checkpoint_path)

# Validation evaluation
val_gen.reset()
val_pred = best_model.predict(val_gen, verbose=1)
val_pred_classes = np.argmax(val_pred, axis=1)
val_true_classes = val_gen.classes

accuracy = np.mean(val_pred_classes == val_true_classes)
print(f"\nFinal Validation Accuracy: {accuracy:.4f}")

# Top-k accuracies
top3_acc = tf.keras.metrics.top_k_categorical_accuracy(
    tf.keras.utils.to_categorical(val_true_classes, num_classes), 
    val_pred, k=3
).numpy().mean()
print(f"Top-3 Accuracy: {top3_acc:.4f}")

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(
    tf.keras.utils.to_categorical(val_true_classes, num_classes), 
    val_pred, k=5
).numpy().mean()
print(f"Top-5 Accuracy: {top5_acc:.4f}")

In [ ]:
# ============================================================================
# TEST PREDICTIONS
# ============================================================================
def predict_test_images():
    test_image_paths = [os.path.join(TEST_DIR, f) 
                       for f in sorted(os.listdir(TEST_DIR)) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not test_image_paths:
        print("No test images found!")
        return None, None
    
    # Create test dataset
    def load_and_preprocess_image(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, IMG_SIZE)
        img = tf.cast(img, tf.float32) / 255.0
        return img
    
    test_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
    test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    # Predict
    predictions = best_model.predict(test_ds, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get class names
    class_names = list(train_gen.class_indices.keys())
    predicted_names = [cat_to_name.get(class_names[i], f"Class_{i}") 
                      for i in predicted_classes]
    
    # Create submission
    submission = pd.DataFrame({
        'filename': [os.path.basename(path) for path in test_image_paths],
        'prediction': predicted_names
    })
    
    submission_path = 'improved_flower_submission.csv'
    submission.to_csv(submission_path, index=False)
    print(f"Submission saved to: {submission_path}")
    
    return submission, predictions

submission_df, test_predictions = predict_test_images()

if submission_df is not None:
    print(f"Predicted {len(submission_df)} test images")
    print("\nPrediction distribution:")
    print(submission_df['prediction'].value_counts().head(10))

print("\n" + "="*50)
print("MODEL IMPROVEMENTS IMPLEMENTED:")
print("="*50)
print("1. ✅ More conservative data augmentation")
print("2. ✅ Transfer learning with EfficientNetB0")
print("3. ✅ Improved learning rate scheduling")
print("4. ✅ Better regularization and dropout")
print("5. ✅ Enhanced monitoring metrics")
print("6. ✅ Fine-tuning phase for transfer learning")
print("7. ✅ Robust data preprocessing")
print("="*50)